# Imports

In [1]:
import os
import fitz
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

model = "gpt-4o-mini"

In [3]:
file_path = '/Users/renatoboemer/code/developer/luthor/data/Memo 2  - Crypto assets disposal - FINISHED .docx'

# Data processing

## Load

In [4]:
import os
import fitz
from docx import Document

def read_file(file_path):
    # Check if file exists
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    # Get the file extension
    _, file_extension = os.path.splitext(file_path)

    # Handle .txt files
    if file_extension.lower() == '.txt':
        return read_txt(file_path)

    # Handle .docx files
    elif file_extension.lower() == '.docx':
        return read_docx(file_path)

    # Handle .pdf files
    elif file_extension.lower() == '.pdf':
        return read_pdf(file_path)

    else:
        raise ValueError(f"Unsupported file extension: {file_extension}")

In [5]:
def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def read_docx(file_path):
    document = Document(file_path)
    full_text = []
    for paragraph in document.paragraphs:
        full_text.append(paragraph.text)
    return '\n'.join(full_text)


def read_pdf(file_path):
    document = fitz.open(file_path)
    all_text = []
    for page in document:
        text = page.get_text()
        all_text.append(text)
    return '\n'.join(all_text)

In [7]:
text = read_file(file_path)
print(text[:350])

Input:
I would like to seek expert advice regarding our company's upcoming transaction involving the disposal of cryptocurrency assets. Specifically, we are planning to sell a substantial amount of Ethereum (ETH) and convert it into stablecoins due to market volatility. Please highlight the tax implications of that. Do we need to formalise the sale


## Process

In [23]:
import re

def text_segmentation(text):
    """
    Split text into smaller, manageable chunks (e.g., paragraphs).

    Args:
        text (str): The full text to be segmented.

    Returns:
        List[str]: A list of segmented text chunks.
    """
    # Split text by double newlines or similar paragraph markers
    segments = re.split(r'\n\s*\n', text)
    
    return segments

In [31]:
from transformers import pipeline

# Load a pre-trained NER pipeline using Hugging Face transformers
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def named_entity_recognition(text):
    """
    Perform named entity recognition using Hugging Face transformers.

    Args:
        text (str): The text for which to identify named entities.

    Returns:
        List[Tuple[str, str]]: A list of entities with their respective types.
    """
    entities = ner_pipeline(text)
    
    # Adjust for entities spanning multiple tokens and merge
    merged_entities = []
    current_entity = None
    for entity in entities:
        word = entity['word']
        
        if word.startswith('##'):
            # If the word is part of a subword token, append to the current entity
            if current_entity is not None:
                current_entity[0] += word[2:]
        else:
            if current_entity is not None:
                merged_entities.append((current_entity[0], current_entity[1]))
            current_entity = [word, entity['entity']]
    
    if current_entity is not None:
        merged_entities.append((current_entity[0], current_entity[1]))
    
    return merged_entities

2024-08-05 19:18:37.278305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [25]:
def tokenize_text(text):
    """
    Tokenize the text into words.

    Args:
        text (str): The text to be tokenized.

    Returns:
        List[str]: A list of tokens (words).
    """
    # Use simple regex to split words; nlp(text) can be used for more advanced tokenization
    tokens = re.findall(r'\b\w+\b', text)
    
    return tokens

In [26]:
def clean_special_characters(text):
    """
    Clean up non-informative special characters or artifacts.

    Args:
        text (str): The text from which to remove special characters.

    Returns:
        str: Cleaned text with unnecessary special characters removed.
    """
    # Remove characters not usually found in legal texts
    cleaned_text = re.sub(r'[^\w\s,.!?;:()-]', '', text)
    
    return cleaned_text

In [27]:
def preserve_structure(text):
    """
    Maintain the document's structural elements, such as headings.

    Args:
        text (str): The text to process for structural preservation.

    Returns:
        str: Text with preserved structure for headings and sections.
    """
    # This can involve wrapping or tagging headings, using markdown for sections
    structured_text = text
    
    # Example: Keep lines starting with capital words as headings
    structured_text = re.sub(r'(?m)^(?=[A-Z])(.+)$', r'## \1', structured_text)
    
    return structured_text

In [28]:
def preprocess_doc(file_path):
    """
    Preprocess a legal document by executing a series of text processing steps.

    Args:
        file_path (str): The path to the legal document text file.

    Returns:
        Tuple: A tuple containing:
            - Original text (str)
            - Segmented text chunks (List[str])
            - Cleaned text (str)
            - Named entities and their types (List[Tuple[str, str]])
            - Tokenized words (List[str])
            - Structured text (str)
    """
    # Load text from the file
    text = read_file(file_path)

    # Split text into segments (paragraphs)
    segments = text_segmentation(text)

    # Clean up non-informative special characters
    cleaned_text = clean_special_characters(text)

    # Perform named entity recognition using Hugging Face transformers
    entities = named_entity_recognition(text)

    # Tokenize the text into words
    tokens = tokenize_text(text)

    # Preserve structural elements, e.g., headings
    structured_text = preserve_structure(text)

    return text, segments, cleaned_text, entities, tokens, structured_text


In [32]:
# Example usage
original_text, segments, cleaned_text, entities, tokens, structured_text = preprocess_doc(file_path)

print("Original Text:", original_text[:200], "...")
print("Segments:", segments[:3])
print("Cleaned Text:", cleaned_text[:200], "...")
print("Entities:", entities[:5])
print("Tokens:", tokens[:10])
print("Structured Text:", structured_text[:200], "...")

NameError: name 'ner_pipeline' is not defined

## Embed

## Create a Search Index

## Query the System